MODEL TRAINING

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

# Load data
df = pd.read_csv("D:\dlproject\dlproj\data\processed\processed_essays.csv")  # Replace with your actual file
essays = df['clean_essay'].astype(str).tolist()
scores = df['domain1_score'].tolist()

# Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(essays, scores, test_size=0.1)

# Tokenize
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)


<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\d'
C:\Users\KEVIN\AppData\Local\Temp\ipykernel_28712\1969151095.py:6: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_csv("D:\dlproject\dlproj\data\processed\processed_essays.csv")  # Replace with your actual file


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\KEVIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\KEVIN\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

c:\Users\KEVIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
import torch

class EssayDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

train_dataset = EssayDataset(train_encodings, train_labels)
val_dataset = EssayDataset(val_encodings, val_labels)


In [4]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


c:\Users\KEVIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [6]:
trainer.train()


  0%|          | 0/11680 [00:00<?, ?it/s]

{'loss': 41.6355, 'grad_norm': 10.494009017944336, 'learning_rate': 4.785958904109589e-05, 'epoch': 0.17}
{'loss': 25.0813, 'grad_norm': 31.94352149963379, 'learning_rate': 4.571917808219178e-05, 'epoch': 0.34}
{'loss': 14.7982, 'grad_norm': 16.629671096801758, 'learning_rate': 4.357876712328767e-05, 'epoch': 0.51}
{'loss': 12.1791, 'grad_norm': 109.85242462158203, 'learning_rate': 4.143835616438356e-05, 'epoch': 0.68}
{'loss': 8.0709, 'grad_norm': 423.0643310546875, 'learning_rate': 3.929794520547945e-05, 'epoch': 0.86}


  0%|          | 0/325 [00:00<?, ?it/s]

{'eval_loss': 7.415451526641846, 'eval_runtime': 20.8794, 'eval_samples_per_second': 62.167, 'eval_steps_per_second': 15.566, 'epoch': 1.0}
{'loss': 6.6905, 'grad_norm': 11.692829132080078, 'learning_rate': 3.715753424657534e-05, 'epoch': 1.03}
{'loss': 5.8205, 'grad_norm': 59.30990982055664, 'learning_rate': 3.501712328767123e-05, 'epoch': 1.2}
{'loss': 7.9913, 'grad_norm': 46.59178161621094, 'learning_rate': 3.287671232876712e-05, 'epoch': 1.37}
{'loss': 6.6568, 'grad_norm': 100.25692749023438, 'learning_rate': 3.073630136986301e-05, 'epoch': 1.54}
{'loss': 4.3876, 'grad_norm': 24.671932220458984, 'learning_rate': 2.8595890410958903e-05, 'epoch': 1.71}
{'loss': 6.7983, 'grad_norm': 487.3072509765625, 'learning_rate': 2.6455479452054793e-05, 'epoch': 1.88}


  0%|          | 0/325 [00:00<?, ?it/s]

...

  0%|          | 0/325 [00:00<?, ?it/s]

{'eval_loss': 4.62291955947876, 'eval_runtime': 21.1466, 'eval_samples_per_second': 61.381, 'eval_steps_per_second': 15.369, 'epoch': 3.0}
{'loss': 3.8675, 'grad_norm': 13.521726608276367, 'learning_rate': 1.1472602739726027e-05, 'epoch': 3.08}
{'loss': 3.3283, 'grad_norm': 24.15824317932129, 'learning_rate': 9.332191780821919e-06, 'epoch': 3.25}
{'loss': 2.5954, 'grad_norm': 18.43943214416504, 'learning_rate': 7.191780821917809e-06, 'epoch': 3.42}
{'loss': 3.3302, 'grad_norm': 312.2174987792969, 'learning_rate': 5.051369863013699e-06, 'epoch': 3.6}
{'loss': 2.6967, 'grad_norm': 17.25446128845215, 'learning_rate': 2.910958904109589e-06, 'epoch': 3.77}
{'loss': 2.6906, 'grad_norm': 40.00421142578125, 'learning_rate': 7.705479452054794e-07, 'epoch': 3.94}


  0%|          | 0/325 [00:00<?, ?it/s]

{'eval_loss': 4.496140480041504, 'eval_runtime': 21.0821, 'eval_samples_per_second': 61.569, 'eval_steps_per_second': 15.416, 'epoch': 4.0}
{'train_runtime': 3227.0557, 'train_samples_per_second': 14.475, 'train_steps_per_second': 3.619, 'train_loss': 7.966116322556587, 'epoch': 4.0}


TrainOutput(global_step=11680, training_loss=7.966116322556587, metrics={'train_runtime': 3227.0557, 'train_samples_per_second': 14.475, 'train_steps_per_second': 3.619, 'total_flos': 1.2290333267042304e+16, 'train_loss': 7.966116322556587, 'epoch': 4.0})

In [7]:
import joblib

# Save the model
joblib.dump(model, "model_weights.pkl")

# Load the model
model = joblib.load("model_weights.pkl")
